# Create H5 Structure for Gasperini Data

This notebook creates a h5 data structure that contains all the information from the Gasperini et al. dataset, including the gene counts matrix, target site-cell matrix, covariates, and guide efficiencies. The h5 structure allows for faster reading of matrices, which improves runtime when training models at scale.

Author: Karthik Guruvayurappan

In [41]:
library(Matrix)
library(rhdf5)

In [2]:
# create matrix to hold Gasperini data
h5createFile('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5')

[1] TRUE

## Create dataframe of cell covariates

In [3]:
# read in covariates dataframe from paper
covariates <- read.table('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale_screen.phenoData.txt.gz')
covariate.names <- read.table('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale.phenoData.colnames.txt')$V1
colnames(covariates) <- covariate.names
head(covariates)

,sample,cell,total_umis,Size_Factor,gene,all_gene,barcode,read_count,umi_count,proportion,guide_count,sample_directory,ko_barcode_file,id,prep_batch,within_batch_chip,within_chip_lane,percent.mito
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1A_1_SI-GA-E2,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,17572,1.0096818,chr10.845_top_two_chr1.11183_top_two_chr1.11293_top_two_chr11.1791_top_two_chr1.12598_top_two_chr1.12806_top_two_chr11.5089_top_two_chr12.283_top_two_chr12.3086_top_two_chr12.4956_top_two_chr13.188_top_two_chr14.1937_top_two_chr14.986_top_two_chr1.5035_top_two_chr15.3069_top_two_chr15.3317_top_two_chr15.3544_top_two_chr16.1866_top_two_chr16.2150_top_two_chr17.3444_top_two_chr17.6206_top_two_chr18.1149_top_two_chr18.252_top_two_chr18.398_second_two_chr18.575_top_two_chr1.8819_top_two_chr18.960_top_two_chr19.2258_top_two_chr19.2871_top_two_chr20.2100_top_two_chr20.282_top_two_chr21.339_top_two_chr2.1667_top_two_chr22.1035_top_two_chr2.2107_top_two_chr22.1271_top_two_chr22.844_top_two_chr2.5796_top_two_chr3.4238_top_two_chr3.767_second_two_chr4.1285_top_two_chr4.1381_top_two_chr4.1723_top_two_chr4.2949_top_two_chr4.3273_top_two_chr5.1426_top_two_chr5.2703_top_two_chr5.4895_top_two_chr6.1373_top_two_chr6.239_top_two_chr6.4987_top_two_chr6.5076_top_two_chr6.5348_top_two_chr7.1421_top_two_chr7.2879_top_two_chr7.4321_top_two_chr7.922_top_two_chr8.1463_top_two_chr8.2152_top_two_chr8.2650_top_two_chr9.1077_top_two_chr9.2443_top_two_chrX.1277_top_two_chrX.600_top_two_EEF2_TSS_MRPL9_TSS_PARK7_TSS,chr10.845_top_two_chr1.11183_top_two_chr1.11293_top_two_chr11.1791_top_two_chr1.12598_top_two_chr1.12806_top_two_chr11.5089_top_two_chr12.283_top_two_chr12.3086_top_two_chr12.4956_top_two_chr13.188_top_two_chr14.1937_top_two_chr14.986_top_two_chr1.5035_top_two_chr15.3069_top_two_chr15.3317_top_two_chr15.3544_top_two_chr16.1866_top_two_chr16.2150_top_two_chr17.3444_top_two_chr17.6206_top_two_chr18.1149_top_two_chr18.252_top_two_chr18.398_second_two_chr18.575_top_two_chr1.8819_top_two_chr18.960_top_two_chr19.2258_top_two_chr19.2871_top_two_chr20.2100_top_two_chr20.282_top_two_chr21.339_top_two_chr2.1667_top_two_chr22.1035_top_two_chr2.2107_top_two_chr22.1271_top_two_chr22.844_top_two_chr2.5796_top_two_chr3.4238_top_two_chr3.767_second_two_chr4.1285_top_two_chr4.1381_top_two_chr4.1723_top_two_chr4.2949_top_two_chr4.3273_top_two_chr5.1426_top_two_chr5.2703_top_two_chr5.4895_top_two_chr6.1373_top_two_chr6.239_top_two_chr6.4987_top_two_chr6.5076_top_two_chr6.5348_top_two_chr7.1421_top_two_chr7.2879_top_two_chr7.4321_top_two_chr7.922_top_two_chr8.1463_top_two_chr8.2152_top_two_chr8.2650_top_two_chr9.1077_top_two_chr9.2443_top_two_chrX.1277_top_two_chrX.600_top_two_EEF2_TSS_MRPL9_TSS_PARK7_TSS,AGAAAGCTCCTCCAGTTCAC_TGATCGCTTTGACTGTGACA_ACAATAAAGAACAGAACACA_GTAAATTGAGACCTCAGGAG_TCTTCCCCCCACCAATAACA_GAGAAAAAAACAATTCAGGC_TCTTAGAGTTCACAGAAGAA_GCTGGGAATTTCTCTCCTGG_AGTGTAACAGAATATCAAAT_ACCCACTGTGACTAGACAAA_AGAAGGATAGAGACTGCTGG_CCAGGCACTTGTGAGAACAA_TGATGGTGTCCCCACCCAAA_GCAGGCCCCATGGATACCCG_AAGGAGTGTGTTCCACACCA_GTACCCTCCCTACCCCCGAG_TGCCTGCTAGAGTCAATAGG_AATGCCAGTTTCCCCCACAG_GCCATTGCTGTAGAGACACT_GCCCAGTCAGAACCCAGGAA_GCACAGATTTACACGCCCGT_GAGAGGCTGCCAGCCCACAG_TGTTCTATATTGCCACCTAG_AGCATCAAATTGCAGAGCAG_ACGAAGAATGAATTGAAGGG_CTGTTTCAGAAAGCTCCCAA_ACTTTGAGCTGCTTCAAGGG_GAGACCTTCCCCCTACCCAG_TTTCCCCGCTGACAGACTGA_ATGACTGCCCCCAGCAGCAA_GGCGCAAAGACAGTGCCAGA_CTCTGACTCACACAACAGGA_GCAAGTTTGCTTTCTCCTGG_TTGAAAGACACATAGCACGA_TCACAGATATTGACTGCCCT_ACAACCCCAAGAACTAGCGG_GCAGCGAAGCTGTTCCACCA_TGCTGCATCCAGATGTTACG_AGACACAGTCAAATGAGGCA_AGCTCTGTCAACCTGCCATG_TCTGCCTGAATGTTTCTCAG_CTGTTTATACCGAGCAGTAG_TGAGCTCCGCCTACACACGG_TTAGTAGAGTGTAGACTGGG_CTCCTTACCCCAGCCAATCG_GGGCCATTACCTTTGCAGAG_GGAGCCCACGACGCTCAAGG_GTTTCCTTATGTATACGTGG_AGAACTCTCATGCGGTCAGG_GAGCTTACTCATGTCCCCCA_CTGACTCTGAGACAGCCAAG_ACCAGAGGAGAGATCAAAGG_GATAAGCAGGAATAAAATCA_AATCACAAGGAAGCAAAGAG_GCAGCTAGATAGATAAGGGG_TTCCATCCTGGGCATGACCA_GGTCGCAGATAGGTGACGGA_GCTCCTGCTTCACTAACCCT_T

In [4]:
# save covariates that were used in previously published model
covariates <- covariates[, c('cell', 'guide_count', 'prep_batch', 'percent.mito')]
head(covariates)

,cell,guide_count,prep_batch,percent.mito
,<chr>,<int>,<chr>,<dbl>
1,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,67,prep_batch_1,0.058786706
2,AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,26,prep_batch_1,0.036086518
3,AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,61,prep_batch_1,0.069823051
4,AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,39,prep_batch_1,0.026186508
5,AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,37,prep_batch_1,0.007991318
6,AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2,57,prep_batch_1,0.022356681


In [5]:
# read in cell cycle scores
s.scores <- read.csv('/iblm/netapp/home/karthik/crisprQTL/gasperini_data/s_scores.csv')
g2m.scores <- read.csv('/iblm/netapp/home/karthik/crisprQTL/gasperini_data/g2m_scores.csv')
colnames(s.scores) <- c('cell', 's.score')
colnames(g2m.scores) <- c('cell', 'g2m.score')
print(head(s.scores))
print(head(g2m.scores))

                              cell      s.score
1 AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2  0.110732311
2 AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2 -0.010290919
3 AAACCTGCAAACAACA-1_1A_1_SI-GA-E2 -0.175860130
4 AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2  0.003057281
5 AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2 -0.144480961
6 AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2  0.026418076
                              cell  g2m.score
1 AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2 -0.1319208
2 AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2 -0.1535426
3 AAACCTGCAAACAACA-1_1A_1_SI-GA-E2 -0.3084879
4 AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2 -0.1574859
5 AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2 -0.2362154
6 AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2 -0.1462899


In [6]:
covariates <- merge(covariates, s.scores, by = 'cell')
covariates <- merge(covariates, g2m.scores, by = 'cell')
head(covariates)

,cell,guide_count,prep_batch,percent.mito,s.score,g2m.score
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,AAACCTGAGAACTGTA-1_2A_4_SI-GA-G5,24,prep_batch_2,0.05442845,0.05704670,0.002865079
2,AAACCTGAGAAGAAGC-1_2B_5_SI-GA-H6,30,prep_batch_2,0.06170906,-0.05632492,0.225004169
3,AAACCTGAGAAGGTTT-1_2A_7_SI-GA-G8,18,prep_batch_2,0.06625193,-0.17070798,0.167475723
4,AAACCTGAGAATAGGG-1_2B_6_SI-GA-H7,34,prep_batch_2,0.01928651,-0.10711479,1.108284663
5,AAACCTGAGAATTCCC-1_1B_2_SI-GA-F3,13,prep_batch_1,0.03646176,-0.15393309,-0.145533466
6,AAACCTGAGACACTAA-1_1B_5_SI-GA-F6,NA,prep_batch_1,0.03699082,-0.05339071,-0.188043139


In [7]:
h5write(covariates, '/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'covariates')

In [8]:
h5ls('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5')

,group,name,otype,dclass,dim
,<chr>,<chr>,<chr>,<chr>,<chr>
0,/,covariates,H5I_DATASET,COMPOUND,207324


In [9]:
covariates <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'covariates')
head(covariates)

,cell,guide_count,prep_batch,percent.mito,s.score,g2m.score
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,AAACCTGAGAACTGTA-1_2A_4_SI-GA-G5,24,prep_batch_2,0.05442845,0.05704670,0.002865079
2,AAACCTGAGAAGAAGC-1_2B_5_SI-GA-H6,30,prep_batch_2,0.06170906,-0.05632492,0.225004169
3,AAACCTGAGAAGGTTT-1_2A_7_SI-GA-G8,18,prep_batch_2,0.06625193,-0.17070798,0.167475723
4,AAACCTGAGAATAGGG-1_2B_6_SI-GA-H7,34,prep_batch_2,0.01928651,-0.10711479,1.108284663
5,AAACCTGAGAATTCCC-1_1B_2_SI-GA-F3,13,prep_batch_1,0.03646176,-0.15393309,-0.145533466
6,AAACCTGAGACACTAA-1_1B_5_SI-GA-F6,NA,prep_batch_1,0.03699082,-0.05339071,-0.188043139


## Read in gene counts matrix and write to h5

In [11]:
gene.counts <- readMM('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale_screen.exprs.mtx')

In [12]:
# read in gene names
genes <- read.table('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale_screen.genes.txt')$V1
rownames(gene.counts) <- genes

In [13]:
# read in cell names
cells <- read.table('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale_screen.cells.txt')$V1
colnames(gene.counts) <- cells

In [14]:
gene.counts <- as.matrix(gene.counts)

In [15]:
h5write(gene.counts, '/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.counts')

Current chunk settings will exceed HDF5's 4GB limit.
Automatically adjusting them to: 13135 x 23170
You may wish to set these to more appropriate values using the 'chunk' argument.



In [16]:
# write cell and gene names to h5 (since they are not automatically written)
h5write(cells, '/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'cell.barcodes')
h5write(genes, '/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.names')

In [17]:
h5ls('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5')

,group,name,otype,dclass,dim
,<chr>,<chr>,<chr>,<chr>,<chr>
0,/,cell.barcodes,H5I_DATASET,STRING,207324
1,/,covariates,H5I_DATASET,COMPOUND,207324
2,/,gene.counts,H5I_DATASET,FLOAT,13135 x 207324
3,/,gene.names,H5I_DATASET,STRING,13135


In [31]:
counts <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.counts')

In [32]:
genes <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'gene.names')
rownames(counts) <- genes

In [39]:
test.count <- counts['ENSG00000237094', ]
test.count

[1] 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [37] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   [73] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [109] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [145] 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [181] 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [217] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [253] 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [289] 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  [325] 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [361] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  [397] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [433] 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [469] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  [505] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [541] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [577] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [613] 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  [649] 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [685] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [721] 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [757] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [793] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [829] 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  [865] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  [901] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [937] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  [973] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1009] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1045] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1081] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 [1117] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1153] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1189] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 [1225] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1261] 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1297] 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1333] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1369] 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 [1405] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1441] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1477] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1513] 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1549] 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1585] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1621] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1657] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1693] 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1729] 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [1765] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [18

In [40]:
cells <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'cell.barcodes')
test.df <- cbind(cells, test.count)
head(test.df)

cells,test.count
AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,0
AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,0
AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,0
AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,1
AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,0
AAACCTGGTAGCGCAA-1_1A_1_SI-GA-E2,0


## Read in guide efficiency values and write to h5

In [46]:
# read in GuideScan output
guidescan.guide.info <- read.csv('/iblm/netapp/home/karthik/GuideScan/Gasperini2019/guidescan_output.csv')
head(guidescan.guide.info)

,Index,gRNA,Chromosome,Start,End,Strand,Num.Off.Targets,Off.Target.Summary,Specificity,Cutting.Efficiency
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>
1,0,CTAAAGCATTGGCTGAGAAGNGG,chr8,23911081,23911103,-,41,2:2 | 3:39,0.13622800,0.565010
2,1,GTAGTTCACATAATCCCTGTNGG,chr4,25698193,25698215,-,55,2:0 | 3:55,0.16592900,0.572492
3,2,AAGTTGACTCTACATAGCAGNGG,chr8,23912565,23912587,+,22,2:1 | 3:21,0.34106700,0.636691
4,3,AATATTCTCCCTCATTCTGGNGG,chr5,12539360,12539382,-,803,2:26 | 3:777,0.00274364,0.619800
5,4,AATCCTCTAATGGACGAAGANGG,chr8,23913057,23913079,-,24,2:0 | 3:24,0.33441500,0.602272
6,5,AGATACCTATGGCCATATAGNGG,chr5,12540099,12540121,+,14,2:0 | 3:14,0.35172300,0.531946


In [47]:
h5write(guidescan.guide.info, '/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'guidescan.output')

In [48]:
h5ls('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5')

,group,name,otype,dclass,dim
,<chr>,<chr>,<chr>,<chr>,<chr>
0,/,cell.barcodes,H5I_DATASET,STRING,207324
1,/,covariates,H5I_DATASET,COMPOUND,207324
2,/,gene.counts,H5I_DATASET,FLOAT,13135 x 207324
3,/,gene.names,H5I_DATASET,STRING,13135
4,/,guidescan.output,H5I_DATASET,COMPOUND,13189


In [51]:
guidescan.output <- h5read('/iblm/netapp/data1/external/Gasperini2019/processed/gasperini_data.h5', 'guidescan.output')
head(guidescan.output)

,Index,gRNA,Chromosome,Start,End,Strand,Num.Off.Targets,Off.Target.Summary,Specificity,Cutting.Efficiency
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>
1,0,CTAAAGCATTGGCTGAGAAGNGG,chr8,23911081,23911103,-,41,2:2 | 3:39,0.13622800,0.565010
2,1,GTAGTTCACATAATCCCTGTNGG,chr4,25698193,25698215,-,55,2:0 | 3:55,0.16592900,0.572492
3,2,AAGTTGACTCTACATAGCAGNGG,chr8,23912565,23912587,+,22,2:1 | 3:21,0.34106700,0.636691
4,3,AATATTCTCCCTCATTCTGGNGG,chr5,12539360,12539382,-,803,2:26 | 3:777,0.00274364,0.619800
5,4,AATCCTCTAATGGACGAAGANGG,chr8,23913057,23913079,-,24,2:0 | 3:24,0.33441500,0.602272
6,5,AGATACCTATGGCCATATAGNGG,chr5,12540099,12540121,+,14,2:0 | 3:14,0.35172300,0.531946


## Read in cell-target site matrix and write to h5